In [1]:
# imports
import requests
import json
import math
import pandas as pd
import spacy

In [39]:
# initial search
url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=California&date1=1770&date2=1963&proxtext=gold+rush&x=28&y=9&dateFilterType=yearRange&rows=20&searchType=basic&format=json'
response = requests.get(url)
raw = response.text
results = json.loads(raw)

In [40]:
results.keys()

dict_keys(['totalItems', 'endIndex', 'startIndex', 'itemsPerPage', 'items'])

In [41]:
# explore items
print(type(results['items']))

<class 'list'>


In [42]:
print(results['items'][0])

{'sequence': 8, 'county': ['Imperial'], 'edition': None, 'frequency': 'Daily', 'id': '/lccn/sn92070146/1926-03-16/ed-1/seq-8/', 'subject': ['California--Imperial County.--fast--(OCoLC)fst01206056', 'California--San Diego County.--fast--(OCoLC)fst01204290', 'Imperial County (Calif.)--Newspapers.', 'San Diego County (Calif.)--Newspapers.'], 'city': ['El Centro'], 'date': '19260316', 'title': 'Imperial Valley press.', 'end_year': 9999, 'note': ['Archived issues are available in digital format as part of the Library of Congress Chronicling America online collection.', 'Latest issue consulted: Oct. 7, 1993.', 'Master negatives are available for duplication from:', 'Published as: Imperial Valley press and El Centro progress, Jan. 16-Oct. 25, 1922.', 'Published combined issues on holidays: The morning post-Imperial Valley press, Nov. 28, 1935 and Dec. 25, 1935. On Sundays published as: The Post-press, Jan. 12, 1936-Oct. 30, 1960; Imperial Valley news-press, Nov. 6, 1960-Feb. 26/27, 1967; Impe

In [31]:
print('totalItems:', results['totalItems'])
print('endIndex:', results['endIndex'])
print('startIndex:', results['startIndex'])
print('itemsPerPage:', results['itemsPerPage'])
print('Length and type of items:', len(results['items']), type(results['items']))

totalItems: 30966
endIndex: 20
startIndex: 1
itemsPerPage: 20
Length and type of items: 20 <class 'list'>


In [58]:
# find total amount of pages
total_pages = math.ceil(results['totalItems'] / results['itemsPerPage'])
print(total_pages)

62


In [59]:
# create empty list for data
data = []

In [60]:
# set search parameters
start_date = '1770'
end_date = '1963'
search_term = 'gold+rush'
state = 'California'

In [61]:
# loop through search results and collect data
for i in range(1, 11):
    url = (f'https://chroniclingamerica.loc.gov/search/pages/results/?state={state}&date1={start_date}'
           f'&date2={end_date}&proxtext={search_term}&x=16&y=8&dateFilterType=yearRange&rows=20'
           f'&searchType=basic&format=json&page={i}')
    response = requests.get(url)
    raw = response.text
    print(f'page {i} status code:', response.status_code)
    results = json.loads(raw)
    items_ = results['items']
    for item_ in items_:
        row_data = {}
        try:
          row_data['title'] = item_['title_normal']
        except:
          row_data['city'] = "none"
        try:
          row_data['city'] = item_['city']
        except:
          row_data['city'] = "none"
        try:
          row_data['date'] = item_['date']
        except:
          row_data['date'] = "none"
        try:
          row_data['raw_text'] = item_['ocr_eng']
        except:
          row_data['raw_text'] = 'none'
    data.append(row_data)

page 1 status code: 200
page 2 status code: 200
page 3 status code: 200
page 4 status code: 200
page 5 status code: 200
page 6 status code: 200
page 7 status code: 200
page 8 status code: 200
page 9 status code: 200
page 10 status code: 200


In [62]:
print(data)

[{'title': 'imperial valley press.', 'city': ['El Centro'], 'date': '19361227', 'raw_text': 'What Mammy Pleasant’s Sinister Power?\nThe Story of Mammy Pleasant Began 88 Years Ago in San Francisco and Continued\nThrough the Pre-Civil War South and Into Houses of 11l Repute, and Ended the Way\nof Most Stories as Mammy Died in Poverty.\nBy Virginia Munn Finch\nIT WAS called the "House of Mystery”!\nStrange, unnatural things happened there—\nintrigue, tragedy, death! But strangest of all\nwas the power of the old Negro woman who\nlived in the house. From this eerie dwelling at\n1661 Octavia Street in San Francisco, the\nshrewd and crafty Negress whom the world\nknew as Mammy Pleasant, ruled not only the\nThomas Bel) family for whom she ostensibly\nworked, but half the politicians in the State of\nCalifornia. Even a governor came under her\nstrange power—a power that she carried with\nher to her grave.\nIntrigue coupled with mystery from time im\nmemorial has captured the attention of peopl

In [63]:
# put data into DataFrame
df = pd.DataFrame.from_dict(data)

In [64]:
df.head()

,title,city,date,raw_text
0,imperial valley press.,[El Centro],19361227,What Mammy Pleasant’s Sinister Power?\nThe Sto...
1,imperial valley press.,[El Centro],19251228,2\nSOCIETY AND CLUE NEWS\nESTHER RALSTON\nAND ...
2,san francisco call.,[San Francisco],19030909,themselves to realize that the men who\nflocke...
3,imperial valley press.,[El Centro],19121204,THE EL CENTRO NATIONAL BANK\n•\nDIRECTORS\nW. ...
4,san francisco call.,[San Francisco],19130309,62\nRochester Begins Like All New Camps\nEVERY...


In [65]:
# convert date column from string to date-time object
df['date'] = pd.to_datetime(df['date'])

In [66]:
df.head()

,title,city,date,raw_text
0,imperial valley press.,[El Centro],1936-12-27,What Mammy Pleasant’s Sinister Power?\nThe Sto...
1,imperial valley press.,[El Centro],1925-12-28,2\nSOCIETY AND CLUE NEWS\nESTHER RALSTON\nAND ...
2,san francisco call.,[San Francisco],1903-09-09,themselves to realize that the men who\nflocke...
3,imperial valley press.,[El Centro],1912-12-04,THE EL CENTRO NATIONAL BANK\n•\nDIRECTORS\nW. ...
4,san francisco call.,[San Francisco],1913-03-09,62\nRochester Begins Like All New Camps\nEVERY...


In [67]:
# sort by date
df = df.sort_values(by='date')

In [68]:
df.head()

,title,city,date,raw_text
2,san francisco call.,[San Francisco],1903-09-09,themselves to realize that the men who\nflocke...
3,imperial valley press.,[El Centro],1912-12-04,THE EL CENTRO NATIONAL BANK\n•\nDIRECTORS\nW. ...
4,san francisco call.,[San Francisco],1913-03-09,62\nRochester Begins Like All New Camps\nEVERY...
6,imperial valley press.,[El Centro],1924-06-06,"Friday, June 6,1924\nWomens Interests\nDR. EDI..."
1,imperial valley press.,[El Centro],1925-12-28,2\nSOCIETY AND CLUE NEWS\nESTHER RALSTON\nAND ...


In [69]:
# write fuction to process text
# load nlp model
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner', 'parser')

def process_text(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    lemmas = [token.lemma_ for token in no_punct]
    lemmas_lower = [lemma.lower() for lemma in lemmas]
    lemmas_string = ' '.join(lemmas_lower)
    return lemmas_string

In [70]:
# apply process_text function
# this may take a few minutes
df['lemmas'] = df['raw_text'].apply(process_text)

In [71]:
# save to csv
df.to_csv(f'{search_term}{start_date}-{end_date}.csv', index=False)